# Dependencies

In [1]:
import requests
import logging
import boto3
from botocore.exceptions import ClientError
from base64 import b64encode
import pprint as pp
from xml.etree import ElementTree as ET

# API credentials
import api_config

C:\Users\mahlo\AppData\Roaming\Python\Python36\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# Vimeo setup

import vimeo

v = vimeo.VimeoClient(
    token=YOUR_ACCESS_TOKEN,
    key=YOUR_CLIENT_ID,
    secret=YOUR_CLIENT_SECRET
)

# Make the request to the server for the "/me" endpoint.
about_me = v.get('/me')

# Make sure we got back a successful response.
assert about_me.status_code == 200

# Load the body's JSON data.
print(about_me.json())

# Pull the videos down from TwentyThree

In [2]:
# Define the authorization endpoints
urlRequestToken = 'http://api.visualplatform.net/oauth/request_token'
urlAccessToken = 'http://api.visualplatform.net/oauth/access_token'
urlAuthorize = 'http://api.visualplatform.net/oauth/authorize'
urlCommunicationInternal = 'http://internal.23video.com/api/method'
urlCommunicationMarketing = 'http://marketing.videomarketingplatform.co/api/method'
urlCommunicationProduct = 'http://accelo-product.videomarketingplatform.co/api/method'

In [3]:
# Define the API Credentials
ttConsumerKey = api_config.ttKey
ttConsumerSecret = api_config.ttConsumerSecret
ttToken = api_config.ttToken
ttTokenSecret = api_config.ttTokenSecret

In [4]:
# Define the request headers
headers = {
    "Content-Type": "application/json",
    "APIKey": f"{ttTokenSecret}"    
}

In [115]:
# Get the count of total videos to download
response = requests.get('http://accelo-product.videomarketingplatform.co/api/photo/list', headers=headers)

targetElement = 1
iterator = 0

for i in response:
    if iterator == targetElement:
        i = str(i)
        splitEquals = i.split("=")
        splitQuote = splitEquals[1].split('\"')
        totalRecords = splitQuote[1]
        print(f"Total records: {totalRecords}")
    
    iterator += 1

Total records: 59


In [114]:
# Get the videos from 23
if int(totalRecords) < 100:
    totalRecords = 100
    
vidNum = 1
page = 1

print("----- Begin -----")
while page <= totalRecords/100:
    
    response = requests.get(f'http://accelo-product.videomarketingplatform.co/api/photo/list?size=100&p={page}', headers=headers)
    root = ET.fromstring(response.content)

    # Iterate over the XML response, and create a new object containing just the desired fields
    videos = {}
    
    vidCount = len(root.getchildren())


    for child in root.iter('*'):
        # Clear the variables for this iteration
        ownerRaw = ''
        category = ''
        title = ''
        description = ''
        link = ''


        if child.tag == 'photo':
            print("---------------------------------------")
            print(f"Reviewing video #{vidNum} of {vidCount}")

            try:
                title = child.attrib['title']
                print(f"Title: {title}")
            except:
                print("No title")

            try:
                ownerRaw = child.attrib['username']
                print(f"Owner: {ownerRaw}")
            except:
                print("No User URL")

            if child.attrib['video_1080p_download'] != '':
                link = child.attrib['video_1080p_download']
                print(f"Download link: {link}")

            try:
                category = child.attrib['album_title']
                print(f"Category: {category}")
            except:
                print("No category")


            # Add the video information to the videos object
            videos[f'video_{vidNum}'] = {
                'Owner': ownerRaw,
                'Category': category,
                'Title': title,
                'Link': link
            }

            # Increment the video counter
            vidNum+=1


        elif child.tag == 'content_text':
            try:
                description = child.attrib['content_text']
                print(f"Description: {description}")
            except:
                continue

        else:
            continue


    print("---------------------------------------") 
    
print("----- Complete -----")

----- Begin -----
---------------------------------------
Reviewing video #1 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #2 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #3 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #4 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f

---------------------------------------
Reviewing video #119 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #120 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #121 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #122 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e/video_1

---------------------------------------
Reviewing video #237 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #238 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #239 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #240 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e/video_1

---------------------------------------
Reviewing video #355 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #356 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #357 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #358 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e/video_1

Owner: christaballingit
Download link: /49543316/52550798/5b16b7a85e008f60ab6294ff82c1fca5/video_1080p/sneak-peek-expenses-webinar-am-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #482 of 59
Title: BPW Office365.mp4
Owner: christaballingit
Download link: /49543312/51976450/7976dedabf1b12732911ed2e73ffcf32/video_1080p/bpw-office365mp4-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #483 of 59
Title: 
Owner: christaballingit
Download link: /49543330/51639125/9425f194032370edd3de62873a5ccaa7/video_1080p/download-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #484 of 59
Title: 
Owner: christaballingit
Download link: /49543322/49912960/c5ba159aa96e2a09d9e4e9e0fc34ee09/video_1080p/download-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #485 of 59
Title: 
Owner: christaballingit
Download link

...

Reviewing video #11805 of 59
Title: Faces & Places | Geoff
Owner: andrewgantan
Download link: /49543328/56095730/fd8ac7ab7b3b778d3fdbed5ec09d6c97/video_1080p/faces-places-geoff-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #11806 of 59
Title: Best Practice Webinar.mov
Owner: christaballingit
Download link: /49543324/53307338/f804d104f8485b38ca0a1200a85fae87/video_1080p/best-practice-webinarmov-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #11807 of 59
Title: 
Owner: christaballingit
Download link: /49543324/53306305/41d8b235946ffc90763283cac8166d56/video_1080p/download-video.mp4
Category: Accelo Best Practices
---------------------------------------
Reviewing video #11808 of 59
Title: BPW_Resource Utilization_6.20
Owner: christaballingit
Category: Accelo Best Practices
---------------------------------------
Reviewing video #11809 of 59
Title: Sneak Peek - Expenses Webinar AM .mp4

---------------------------------------
Reviewing video #11919 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #11920 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #11921 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #11922 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e

---------------------------------------
Reviewing video #12037 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12038 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12039 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12040 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e

---------------------------------------
Reviewing video #12096 of 59
Title: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12097 of 59
Title: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Download link: /49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12098 of 59
Title: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #12099 of 59
Title: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56272346/26c5d1d736f4c18ff85b4028ddf69f0e

KeyboardInterrupt: 

# Vimeo